In [ ]:
#!pip3 install datasets

In [ ]:
#!pip3 install transformers -U;

In [ ]:
#!pip3 install accelerate -U

In [32]:
import numpy as np
import pandas as pd
import seaborn as sns

from sklearn.metrics import classification_report
from sklearn.model_selection import cross_validate
from sklearn.metrics import mean_squared_error
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score

import matplotlib.pyplot as plt

import torch
import tensorflow as tf

from sklearn.model_selection import train_test_split

import datasets
from datasets import Dataset, DatasetDict

from transformers import AutoTokenizer
from transformers import AutoModelForSequenceClassification
from transformers import TrainingArguments
from transformers import Trainer

In [ ]:
# !git clone https://github.com/cse151a-DrugReviewAnalysis/DrugReviewAnalysis.git

In [2]:
# Get dataset for working locally
df_train = pd.read_csv("drugsTrain_processed_with_usefulCount.csv", sep="\t")
df_test = pd.read_csv("drugsTest_processed_with_usefulCount.csv", sep="\t")

In [ ]:
# Get dataset when working on colab
# df_train = pd.read_csv("DrugReviewAnalysis/Model 3/drugsTrain_processed_with_usefulCount.csv", sep="\t")
# df_test = pd.read_csv("DrugReviewAnalysis/Model 3/drugsTest_processed_with_usefulCount.csv", sep="\t")

In [3]:
df_train = df_train.sample(frac=0.7, random_state=42).reset_index()
df_test = df_test.sample(frac=0.7, random_state=42).reset_index()

In [4]:
df_train.shape, df_test.shape

((111649, 8), (37240, 8))

In [5]:
df_val, df_test_new = train_test_split(df_test, test_size=0.6, random_state=42)

In [6]:
ds_train = Dataset.from_pandas(df_train[["processed_review", "rating"]])
ds_val = Dataset.from_pandas(df_val.reset_index()[["processed_review", "rating"]])
ds_test = Dataset.from_pandas(df_test_new.reset_index()[["processed_review", "rating"]])

In [7]:
ds = DatasetDict()

ds["train"] = ds_train
ds["validation"] = ds_val
ds["test"] = ds_test

In [8]:
ds

DatasetDict({
    train: Dataset({
        features: ['processed_review', 'rating'],
        num_rows: 111649
    })
    validation: Dataset({
        features: ['processed_review', 'rating'],
        num_rows: 14896
    })
    test: Dataset({
        features: ['processed_review', 'rating'],
        num_rows: 22344
    })
})

In [9]:
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

In [10]:
def preprocess(samples):
  text = samples["processed_review"]
  encoding = tokenizer(text, padding="max_length", truncation=True, max_length=128, return_attention_mask=True)
  encoding["labels"] = samples["rating"]
  return encoding

In [11]:
encoded_dataset = ds.map(preprocess, batched=True, remove_columns=ds['train'].column_names)

Map: 100%|██████████| 22344/22344 [00:02<00:00, 8521.21 examples/s]


In [12]:
encoded_dataset.set_format("torch")

In [13]:
model = AutoModelForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=1)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [14]:

args = TrainingArguments(
    output_dir="/bert-regression", # for local
    # output_dir = "DrugReviewAnalysis/Model 3/bert-regression", # for colab
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=64,
    per_device_eval_batch_size=64,
    num_train_epochs=5,
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model="loss",
)

In [36]:
def compute_metrics(eval_pred):
  predictions, labels = eval_pred
  mse = mean_squared_error(labels, predictions)
  pred_modified = np.where(predictions < 10.0, np.round(predictions), 10.0)
  print(labels)
  acc = accuracy_score(labels, pred_modified)
  f1 = f1_score(labels, pred_modified, average="macro")
  return {"mse": mse, "accuracy-approximation": acc, "f1-approximation": f1}

In [17]:
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=encoded_dataset["train"],
    eval_dataset=encoded_dataset["validation"],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

c:\Users\jason\AppData\Local\Programs\Python\Python311\Lib\site-packages\accelerate\accelerator.py:432: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


In [18]:
trainer.train()

  6%|▌         | 500/8725 [03:10<52:00,  2.64it/s] 

{'loss': 7.5869, 'grad_norm': 64.28229522705078, 'learning_rate': 1.8853868194842408e-05, 'epoch': 0.29}


 11%|█▏        | 1000/8725 [06:22<48:54,  2.63it/s] 

{'loss': 3.6527, 'grad_norm': 44.7933349609375, 'learning_rate': 1.7707736389684814e-05, 'epoch': 0.57}


 17%|█▋        | 1500/8725 [09:34<46:03,  2.61it/s]  

...

                                                   
100%|██████████| 8725/8725 [57:54<00:00,  3.12it/s]

{'eval_loss': 2.3104214668273926, 'eval_mse': 2.3104214668273926, 'eval_runtime': 31.1474, 'eval_samples_per_second': 478.242, 'eval_steps_per_second': 7.481, 'epoch': 5.0}


100%|██████████| 8725/8725 [57:57<00:00,  2.51it/s]

{'train_runtime': 3477.146, 'train_samples_per_second': 160.547, 'train_steps_per_second': 2.509, 'train_loss': 2.26018678331785, 'epoch': 5.0}


TrainOutput(global_step=8725, training_loss=2.26018678331785, metrics={'train_runtime': 3477.146, 'train_samples_per_second': 160.547, 'train_steps_per_second': 2.509, 'train_loss': 2.26018678331785, 'epoch': 5.0})

In [19]:
trainer.evaluate()

100%|██████████| 233/233 [00:32<00:00,  7.11it/s]


{'eval_loss': 2.3104214668273926,
 'eval_mse': 2.3104214668273926,
 'eval_runtime': 33.1678,
 'eval_samples_per_second': 449.111,
 'eval_steps_per_second': 7.025,
 'epoch': 5.0}

In [20]:
# local
trainer.save_model("./bert-regression") 

# colab
# trainer.save_model("DrugReviewAnalysis/Model 3/bert-regression")

In [ ]:
# local
# model = AutoModelForSequenceClassification.from_pretrained("bert-regression")

# colab
# model = AutoModelForSequenceClassification.from_pretrained("DrugReviewAnalysis/Model 3/bert-regression")

# trainer = Trainer(
#     model=model,
#     args=args,
#     train_dataset=encoded_dataset["train"],
#     eval_dataset=encoded_dataset["validation"],
#     tokenizer=tokenizer,
#     compute_metrics=compute_metrics
# )

In [21]:
trainer.evaluate(eval_dataset=encoded_dataset["test"])

100%|██████████| 350/350 [00:47<00:00,  7.42it/s]


{'eval_loss': 2.30594801902771,
 'eval_mse': 2.30594801902771,
 'eval_runtime': 47.3482,
 'eval_samples_per_second': 471.908,
 'eval_steps_per_second': 7.392,
 'epoch': 5.0}